# Add the .atm spectr in the database from folder

import

In [216]:
import os
from turtle import color
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from turtle import color
import seaborn as sns
from sklearn.linear_model import LinearRegression 
from pathlib import Path
import plotly.express as px
import os

select the folder


In [217]:
folder_name = 'D:\work\\1315\Коэф перехода 5178'
spectr_list = [i for i in os.listdir(folder_name) if i.endswith('.ats')]

# add the comments
coments = 'методика поверки, испытания Белгим,'

In [218]:
# functions for reading spectra
def defin_nucl(name):
    a = ''
    if 'cs' in name.lower(): a = 'Cs-137'
    elif 'sr' in name.lower(): a = 'Sr-90'
    elif '1C0' in name.lower(): a = 'Sr-90'
    elif '3C0' in name.lower(): a = 'Sr-90'
    elif '4C0' in name.lower(): a = 'Sr-90'
    elif '5C0' in name.lower(): a = 'Sr-90', 
    elif ('fon' in name.lower()) or ('фон' in name.lower()): a = 'background'
    elif ('phon' in name.lower()): a = 'background'
    return a


def read_spectr(file_path):

    nuclid = defin_nucl(file_path)
    E_cal = []
    R_cal = []
    Rest = []
    sp_gamma = []
    with open (file_path) as spec:
        n_line = 0
        for line in spec:
            if line.startswith('TIME'): time = int(line[line.index('=')+2 :-1])
            elif line.startswith('GEOMETRY'): geom = line[line.index('=')+2 :-1]
            elif line.startswith('WEIGHT'): weight = float(line[line.index('=')+2 :-1])
            elif line.startswith('DATE '): date = line[line.index('=')+2 :-1]
            elif line.startswith('DU_TYPE'): s_n = line[line.index('=')+2 :-1]
            
            n_line += 1
            if n_line in range(48,1071): 
                E_cal.append(float(line.strip()))
            if n_line in range(2098,3121): 
                R_cal.append(float(line.strip()))
            if n_line in range(3123,4145): 
                sp_gamma.append(float(line.strip()))
            if n_line in range(1073,2096): 
                Rest.append(float(line.strip()))
    sp_gamma.append(0)
    dict_d_ats = {'time':time,
                'serial_number':s_n, 
                'geometry':geom, 
                'weight':weight, 
                'date':date, 
                'sp_beta':[Rest], 
                'Energy':[E_cal], 
                'sp_gamma':[sp_gamma], 
                'R_cal':[R_cal], 
                'coments': coments,
                'nuclid': nuclid
                }
    df_ats = pd.DataFrame(dict_d_ats)
    return df_ats


In [219]:
# read database
db = pd.read_csv('1315_database.csv')


In [226]:
for i in spectr_list:

    df_from_ats = read_spectr(folder_name + '\\' + i)

    # forming name of spectr
    if i.endswith(str(int(df_from_ats['serial_number']))):
        Name = i[:-4] 
    else:
        Name = i[:-4] + '.' + df_from_ats['serial_number']
    df_from_ats['name'] = Name

    # checking for repetitions
    if Name[0] in db['name'].values:
        db[db['name'] == i] = df_from_ats.values
    else:
        db = pd.concat([db, df_from_ats], ignore_index=True)



In [221]:
db[['name', 'nuclid']]

,name,nuclid
0,1C0_2125 без держателя.5175,Sr-90
1,"1C0_2125 без держателя 0,1.5175",Sr-90
2,"1C0_2125 без держателя 0,2.5175",Sr-90
3,"1C0_2125 без держателя 0,03.5175",Sr-90
4,"1C0_2125 без держателя 0,5.5175",Sr-90
...,...,...
494,V02_Phon.5176,background
495,V05_Phon.5176,background
496,3С0 2127 без держ + оргстекло.5178.5178,
497,3С0 2127 без держ + сосуд 003.5178.5178,


In [222]:
# writing database
db.to_csv('1315_database.csv', index=False)
db.name

0                  1C0_2125 без держателя.5175
1              1C0_2125 без держателя 0,1.5175
2              1C0_2125 без держателя 0,2.5175
3             1C0_2125 без держателя 0,03.5175
4              1C0_2125 без держателя 0,5.5175
                        ...                   
494                              V02_Phon.5176
495                              V05_Phon.5176
496    3С0 2127 без держ + оргстекло.5178.5178
497    3С0 2127 без держ + сосуд 003.5178.5178
498                3С0 2127 без держ.5178.5178
Name: name, Length: 499, dtype: object